In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import shutil
from math import ceil
from time import time
%matplotlib inline

In [2]:
#os.environ["CUDA_VISIBLE_DEVICES"]="0"
tf.reset_default_graph()

In [3]:
def FindAgeRange(csvfile):
    df = pd.read_csv(csvfile)
    return (df['age'].min(), df['age'].max())

In [4]:
AGE_CSV = "sports_clean_age_list_extreme_biased.csv"
GEN_CSV = "gen_list_extreme_biased.csv"

In [5]:
# Model Parameters
IMG_SIZE      = [96, 96]
BATCH_SIZE    = 32
ITERS         = 10
EPOCHS        = 50
LEARNING_RATE = 0.001
SPLIT         = {"train" : 0.7, "validation": 0.1, "test": 0.2}

# For age classification
AGE_MIN, AGE_MAX = FindAgeRange(AGE_CSV)
BIN_SIZE         = 4
NUM_BINS         = (AGE_MAX-AGE_MIN)//BIN_SIZE
NUM_CLASSES_AGE  = NUM_BINS+1

# For gender classification
NUM_CLASSES_GEN = 2

# The weight of confusion loss
ALPHA = 2

In [6]:
def ReadImages(elem):
    filename = elem["name"][0]
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image_decoded, tf.uint8)
    image = tf.image.resize_images(image, IMG_SIZE)
    elem["name"] = image
    return elem

In [7]:
def AgeLabelFunc(elem):
    label = elem["age"][0]
    label = (label-AGE_MIN)//BIN_SIZE
    elem["age"] = label
    return elem

In [8]:
def GenderLabelFunc(elem):
    elem["gender"] = elem["gender"][0]
    return elem

In [9]:
def SetDatasetParams(dataset, size):
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(1)
    dataset = dataset.shuffle(size)
    dataset = dataset.repeat()
    return dataset

In [10]:
def CreateDataset(csvfile, split, label_func):
    
    # Get the size of the dataset
    data = pd.read_csv(csvfile)
    size = data.shape[0]

    # Create a train, validation, test split 
    train_size      = int(split["train"]*size)
    validation_size = int(split["validation"]*size)
    test_size       = int(split["test"]*size)

    full_data       = tf.contrib.data.make_csv_dataset(csvfile, batch_size=1)
    full_data       = full_data.map(label_func, num_parallel_calls=100)
    full_data       = full_data.map(ReadImages, num_parallel_calls=100)

    train_data      = full_data.take(train_size)
    train_data      = SetDatasetParams(train_data, train_size)
    
    validation_data = full_data.skip(train_size)
    test_data       = validation_data.skip(validation_size)
    test_data       = SetDatasetParams(test_data, test_size)
    
    validation_data = validation_data.take(validation_size)
    validation_data = SetDatasetParams(validation_data, validation_size)

    return train_data, validation_data, test_data, size

In [11]:
train_age, validation_age, test_age, size_age = CreateDataset(AGE_CSV, SPLIT, AgeLabelFunc)

Instructions for updating:
Use `tf.data.experimental.make_csv_dataset(...)`.


In [12]:
train_gen, validation_gen, test_gen, size_gen  = CreateDataset(GEN_CSV, SPLIT, GenderLabelFunc) 

In [13]:
def GetInitOps(train, validation, test):
    init_ops = {}
    it = tf.data.Iterator.from_structure(train.output_types, train.output_shapes)
    init_ops["train"] = it.make_initializer(train)
    init_ops["validation"] = it.make_initializer(validation)
    init_ops["test"] = it.make_initializer(test)
    return init_ops, it

In [14]:
init_ops_age_data, it_age_data = GetInitOps(train_age, validation_age, test_age)
init_ops_gen_data, it_gen_data = GetInitOps(train_gen, validation_gen, test_gen)

In [15]:
def GetNumBatchesPerEpoch(size):
    num_batches = {}
    num_batches["train"]      = int((size*SPLIT["train"])//BATCH_SIZE)
    num_batches["validation"] = int((size*SPLIT["validation"])//BATCH_SIZE)
    num_batches["test"]       = int((size*SPLIT["validation"])//BATCH_SIZE)
    return num_batches

In [16]:
num_batches_age = GetNumBatchesPerEpoch(size_age)
num_batches_gen = GetNumBatchesPerEpoch(size_gen)

In [17]:
def GetAgeFromBin(bin_val):
    return AGE_MIN + (bin_val*BIN_SIZE)

In [18]:
def ShowImages(imgs, age, GetLabel):
    high = imgs.eval().shape[0]
    idx = np.random.randint(low=1, high=high, size=2)
    plt.figure(figsize=(8, 8))

    plt.subplot(121)
    curr_img = np.asarray(imgs[idx[0]].eval(), dtype=np.uint8)
    curr_lbl = age[idx[0]].eval()

    plt.imshow(curr_img)
    plt.title("(Label: {lbl})".format(lbl=GetLabel(curr_lbl)))

    plt.subplot(122)
    curr_img = np.asarray(imgs[idx[1]].eval(), dtype=np.uint8)
    curr_lbl = age[idx[1]].eval()
    plt.imshow(curr_img, cmap="gray")
    plt.title("(Label: {lbl})".format(lbl=GetLabel(curr_lbl)))

In [19]:
def CheckAgeDataset(init_op, it):
    with tf.Session() as sess:
        sess.run(init_op)
        elem = it.get_next()
        ShowImages(elem["name"], elem["age"], GetAgeFromBin)

In [20]:
#CheckAgeDataset(init_ops_age_data["train"], it_age_data)

In [21]:
def GetGenFromBin(bin_val):
    if bin_val == 1:
        return "male"
    else:
        return "female"

In [22]:
def CheckGenDataset(init_op, it):
    with tf.Session() as sess:
        sess.run(init_op)
        elem = it.get_next()
        ShowImages(elem["name"], elem["gender"], GetGenFromBin)

In [23]:
#CheckGenDataset(init_ops_gen_data["train"], it_gen_data)

In [24]:
def conv2d(x, W, b, strides=1):
    strides_ = [1, strides, strides, 1]
    x = tf.nn.conv2d(x, W, strides=strides_, padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [25]:
def maxpool2d(x, k=2):
    ksize_ = [1, k, k, 1]
    strides_ = [1, k, k, 1]
    return tf.nn.max_pool(x, ksize_, strides_, padding='SAME')

In [26]:
VAL  = IMG_SIZE[0]
FLAT = ceil(ceil(ceil(VAL/2)/2))/2

In [27]:
weights = {
    'wc1': tf.get_variable('W0', shape=(3, 3, 3, 32), initializer=tf.contrib.layers.xavier_initializer()),
    'wc2': tf.get_variable('W1', shape=(3, 3, 32, 64), initializer=tf.contrib.layers.xavier_initializer()),
    'wc3': tf.get_variable('W2', shape=(3, 3, 64, 128), initializer=tf.contrib.layers.xavier_initializer()),
    'wd1': tf.get_variable('W3', shape=(FLAT*FLAT*128, 128), initializer=tf.contrib.layers.xavier_initializer()),
    'age': tf.get_variable('W4', shape=(128, NUM_CLASSES_AGE), initializer=tf.contrib.layers.xavier_initializer()),
    'gen': tf.get_variable('W5', shape=(128, NUM_CLASSES_GEN), initializer=tf.contrib.layers.xavier_initializer())
}

biases = {
    'bc1': tf.get_variable('B0', shape=(32), initializer=tf.contrib.layers.xavier_initializer()),
    'bc2': tf.get_variable('B1', shape=(64), initializer=tf.contrib.layers.xavier_initializer()),
    'bc3': tf.get_variable('B2', shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
    'bd1': tf.get_variable('B3', shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
    'age': tf.get_variable('B4', shape=(NUM_CLASSES_AGE), initializer=tf.contrib.layers.xavier_initializer()),
    'gen': tf.get_variable('B5', shape=(NUM_CLASSES_GEN), initializer=tf.contrib.layers.xavier_initializer())
}

In [28]:
# Setup for saving the model with best validation
saver = tf.train.Saver()
save_dir = 'checkpoints-alpha-2-extreme-bias/'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
save_path = os.path.join(save_dir, 'best_validation')

In [29]:
graph_dir = 'graphs-alpha-2-extreme-bias/'

if os.path.exists(graph_dir):
    shutil.rmtree(graph_dir, ignore_errors=True)

os.makedirs(graph_dir)

In [30]:
def conv_net(x, weights, biases, task):
    
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2) 
    
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)
    
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    conv3 = maxpool2d(conv3, k=2)
    
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    
    out_gen = tf.add(tf.matmul(fc1, weights['gen']), biases['gen'])
    out_age = tf.add(tf.matmul(fc1, weights['age']), biases['age'])
    
    return out_gen, out_age

In [31]:
# PRIMARY TASK
# ============

# Running one batch from the age dataset through the network
# Computing the following losses:
# Classification loss (loss_age)
# Confusion loss (loss_confusion)

elem = it_age_data.get_next()
x_age = elem["name"]
y_age = elem["age"]
pred_gen, pred_age = conv_net(x_age, weights, biases, task="primary")

loss_age = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=pred_age, labels=y_age), name="loss_age")

vec = tf.constant([0.5, 0.5])
multiply = tf.shape(x_age)[0]
uniform_gender_lbl = tf.reshape(multiply, [1, ])

gen_labels_op = tf.reshape(tf.tile(vec, uniform_gender_lbl), [ uniform_gender_lbl[0], tf.shape(vec)[0]])

with tf.control_dependencies([gen_labels_op]):
    loss_confusion = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred_gen, labels=gen_labels_op), name="loss_confusion")

# Uniform gender labels
# =====================

# We need to compute cross_entropy between 
# uniform_gender distribution and predictions
# uniform_gender = BATCH_SIZE rows of [0.5, 0.5]

optimizer_primary = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE).minimize(loss_age + ALPHA*loss_confusion)

correct_prediction_age = tf.equal(tf.argmax(pred_age, 1), tf.cast(y_age, tf.int64))
accuracy_age = tf.reduce_mean(tf.cast(correct_prediction_age, tf.float32), name="accuracy_age")

In [32]:
# SECONDARY TASK
# ==============

# Running one batch from the gender dataset through the network
# Computing the following losses:
# Classification loss (loss_gen)

elem = it_gen_data.get_next()
x_gen = elem["name"]
y_gen = elem["gender"]
pred_gen, _ = conv_net(x_gen, weights, biases, task="secondary")
loss_gen = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=pred_gen, labels=y_gen))
optimizer_gen = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE).minimize(loss_gen)

correct_prediction_gen = tf.equal(tf.argmax(pred_gen, 1), tf.cast(y_gen, tf.int64))
accuracy_gen = tf.reduce_mean(tf.cast(correct_prediction_gen, tf.float32))

In [33]:
# Best validation accuracy seen so far.
best_validation_accuracy_primary   = -1

# Validation accuracy thresholds for
# early stopping training 
age_accuracy_threshold = 0.90
gen_accuracy_threshold = 0.98

# Iteration-number for last improvement to validation accuracy.
last_improvement = 0

# Stop optimization if no improvement found in this many iterations.
require_improvement = 1000

In [34]:
# TensorFlow Summaries
# Create tf.summary() objects for all the 
# variables we would like to summarize

# Summarising the training losses
loss_age_summary = tf.summary.scalar('loss_age_summary', loss_age)
loss_gen_summary = tf.summary.scalar('loss_gen_summary', loss_gen)
loss_confusion_summary = tf.summary.scalar('loss_confusion_summary', loss_confusion)

# Summarising the training accuracies
acc_age_summary = tf.summary.scalar('acc_age_summary', accuracy_age)
acc_gen_summary = tf.summary.scalar('acc_gen_summary', accuracy_gen)

# Check if the weights are getting updated properly
age_final_layer_summary = tf.summary.histogram('age_final_layer_summary', weights['age'])
gen_final_layer_summary = tf.summary.histogram('gen_final_layer_summary', weights['gen'])

In [35]:
summaries = [acc_age_summary, loss_age_summary, loss_confusion_summary, age_final_layer_summary, gen_final_layer_summary]
train_primary_summaries = tf.summary.merge(summaries)

summaries = [acc_gen_summary, loss_gen_summary, age_final_layer_summary, gen_final_layer_summary]
train_secondary_summaries = tf.summary.merge(summaries)

In [36]:
# Global step values for summaries
train_primary_step = 0
train_secondary_step = 0

validation_primary_step = 0
validation_secondary_step = 0

In [37]:
# Running this op will initialize weights and biases
init_op = tf.global_variables_initializer()

In [38]:
def SaveModel(sess, acc):
    global best_validation_accuracy_primary
    
    if acc > best_validation_accuracy_primary:
        best_validation_accuracy_primary = acc
        print("[INFO] [PRIMARY] Saving the model parameters")
        saver.save(sess=sess, save_path=save_path)
        return
            
    print("[INFO] [PRIMARY] No improvement in validation accuracy.")

In [39]:
def ValidateSecondaryTask(sess):
    print("[INFO] [SECONDARY] Running validation for the secondary task.")
    
    global validation_secondary_step
    sess.run(init_ops_gen_data["validation"])
    num_batches = num_batches_gen["validation"]
    
    total_loss = 0
    total_acc  = 0
    for _ in range(num_batches):
        loss, acc = sess.run([loss_gen, accuracy_gen])
        total_loss += loss
        total_acc  += acc

    loss = total_loss/num_batches
    acc = total_acc/num_batches
    
    validation_secondary_step += 1
    summary = tf.Summary()
    summary.value.add(tag='validation_secondary_summary', simple_value = acc)
    writer.add_summary(summary, validation_secondary_step)
    
    print('[INFO] [SECONDARY] Validation Loss: {:.4f}, Accuracy: {:.2f}: '.format(loss, acc))
    
    return acc

In [40]:
def ValidatePrimaryTask(sess):
    print("[INFO] [PRIMARY] Running validation for the primary task.")
    
    global validation_primary_step
    sess.run(init_ops_age_data["validation"])
    num_batches = num_batches_age["validation"]
    
    total_loss = 0
    total_acc  = 0
    for _ in range(num_batches):
            loss, acc = sess.run([loss_age, accuracy_age])
            total_loss += loss
            total_acc  += acc

    loss = total_loss / num_batches
    acc  = total_acc/ num_batches
    
    validation_primary_step += 1
    summary = tf.Summary()
    summary.value.add(tag='validation_primary_summary', simple_value = acc)
    writer.add_summary(summary, validation_primary_step)
    
    print('[INFO] [PRIMARY] Validation Loss: {:.4f}, Accuracy: {:.2f}: '.format(loss, acc))
    
    SaveModel(sess, acc)
    
    return acc

In [41]:
def TestPrimaryTask(sess):
    print("\n[INFO] [PRIMARY] Testing on the primary task.")
    
    sess.run(init_ops_age_data["test"])
    num_batches = num_batches_age["test"]

    total_loss = 0
    total_acc  = 0
    for _ in range(num_batches):
        loss, acc = sess.run([loss_age, accuracy_age])
        total_loss += loss
        total_acc  += acc

    loss = total_loss / num_batches
    acc  = total_acc/ num_batches
    print('[INFO] [PRIMARY] Test Loss: {:.4f}, Accuracy: {:.2f}: '.format(loss, acc))

In [42]:
def TestSecondaryTask(sess):
    print("\n[INFO] [SECONDARY] Testing the secondary task.")

    sess.run(init_ops_gen_data["test"])
    num_batches = num_batches_gen["test"]
    
    total_loss = 0
    total_acc  = 0
    for _ in range(num_batches):
        loss, acc = sess.run([loss_gen, accuracy_gen])
        total_loss += loss
        total_acc  += acc

    loss = total_loss/num_batches
    acc  = total_acc/num_batches
    print('[INFO] [SECONDARY] Test Loss: {:.4f}, Accuracy: {:.2f}: '.format(loss, acc))

In [43]:
def TrainPrimaryTask(sess, iter_num):
    print("\n[INFO] [PRIMARY] Training for the primary task.")
    
    global train_primary_step
    sess.run(init_ops_age_data["train"])
    num_batches = num_batches_age["train"]
    
    for epoch in range(EPOCHS):
    
        total_loss = 0
        for i in range(num_batches):
            var = [loss_age, loss_confusion, accuracy_age, optimizer_primary]
            loss, conf, acc, _ = sess.run(var)
            total_loss += loss

        avg_loss = total_loss / num_batches

        train_primary_step += 1
        summary = sess.run(train_primary_summaries)
        writer.add_summary(summary, train_primary_step)

        print("[INFO] [PRIMARY] Iter: {}, Epoch: {}, Loss: {:.4f}, Accuracy: {:.2f}".format(iter_num, epoch,  avg_loss, acc))
    
        acc = ValidatePrimaryTask(sess)
        if acc > age_accuracy_threshold:
            print("[INFO] [PRIMARY] Fully optimized, stopping early.")
            break

In [44]:
def TrainSecondaryTask(sess, iter_num):
    print("\n[INFO] [SECONDARY] Training for the secondary task.")
    
    global train_secondary_step
    sess.run(init_ops_gen_data["train"])
    num_batches = num_batches_gen["train"]
    
    for i in range(EPOCHS):
        
        total_loss = 0
        for _ in range(num_batches):
            var = [optimizer_gen, loss_gen, accuracy_gen]
            _, loss, acc = sess.run(var)
            total_loss += loss
        
        avg_loss = total_loss / num_batches
        
        train_secondary_step += 1
        summary = sess.run(train_secondary_summaries)
        writer.add_summary(summary, train_secondary_step)
        
        print("[INFO] [SECONDARY] Iter: {}, Epoch: {}, Loss: {:.4f}, Accuracy: {:.2f}".format(iter_num, i,  avg_loss, acc))
            
        acc = ValidateSecondaryTask(sess)
        if acc > gen_accuracy_threshold:
            print("[INFO] [SECONDARY] Fully optimized, stopping early.")
            break

In [45]:
def JLU(sess):
    print("[INFO] JLU training for gender agnostic age prediction.")
    sess.run(init_op)

    for iter_num in range(ITERS):
        TrainSecondaryTask(sess, iter_num)
        TrainPrimaryTask(sess, iter_num)

    # Restore the best validation weights before testing
    saver.restore(sess=sess, save_path=save_path)
    
    # This would hopefully be bad
    TestSecondaryTask(sess)
    
    # This would hopefully be good
    TestPrimaryTask(sess)

In [46]:
t1 = time()

with tf.Session() as sess:
    writer = tf.summary.FileWriter(graph_dir, sess.graph)
    JLU(sess)

t2 = time()
print("[INFO] JLU training for gender agnostic age prediction took {:.2f} minutes".format((t2-t1)/60))

[INFO] JLU training for gender agnostic age prediction.

[INFO] [SECONDARY] Training for the secondary task.
[INFO] [SECONDARY] Iter: 0, Epoch: 0, Loss: 25.1462, Accuracy: 0.91
[INFO] [SECONDARY] Running validation for the secondary task.
[INFO] [SECONDARY] Validation Loss: 0.4388, Accuracy: 0.80: 
[INFO] [SECONDARY] Iter: 0, Epoch: 1, Loss: 0.1697, Accuracy: 1.00
[INFO] [SECONDARY] Running validation for the secondary task.
[INFO] [SECONDARY] Validation Loss: 0.1198, Accuracy: 0.96: 
[INFO] [SECONDARY] Iter: 0, Epoch: 2, Loss: 0.0433, Accuracy: 1.00
[INFO] [SECONDARY] Running validation for the secondary task.
[INFO] [SECONDARY] Validation Loss: 0.0401, Accuracy: 0.99: 
[INFO] [SECONDARY] Fully optimized, stopping early.

[INFO] [PRIMARY] Training for the primary task.
[INFO] [PRIMARY] Iter: 0, Epoch: 0, Loss: 2.6369, Accuracy: 0.22
[INFO] [PRIMARY] Running validation for the primary task.
[INFO] [PRIMARY] Validation Loss: 2.4032, Accuracy: 0.16: 
[INFO] [PRIMARY] Saving the model par

[INFO] [PRIMARY] Validation Loss: 0.8250, Accuracy: 0.80: 
[INFO] [PRIMARY] Saving the model parameters
[INFO] [PRIMARY] Iter: 0, Epoch: 32, Loss: 0.2372, Accuracy: 1.00
[INFO] [PRIMARY] Running validation for the primary task.
[INFO] [PRIMARY] Validation Loss: 0.7904, Accuracy: 0.76: 
[INFO] [PRIMARY] No improvement in validation accuracy.
[INFO] [PRIMARY] Iter: 0, Epoch: 33, Loss: 0.2306, Accuracy: 1.00
[INFO] [PRIMARY] Running validation for the primary task.
[INFO] [PRIMARY] Validation Loss: 1.0811, Accuracy: 0.78: 
[INFO] [PRIMARY] No improvement in validation accuracy.
[INFO] [PRIMARY] Iter: 0, Epoch: 34, Loss: 0.3465, Accuracy: 1.00
[INFO] [PRIMARY] Running validation for the primary task.
[INFO] [PRIMARY] Validation Loss: 1.0943, Accuracy: 0.72: 
[INFO] [PRIMARY] No improvement in validation accuracy.
[INFO] [PRIMARY] Iter: 0, Epoch: 35, Loss: 0.3114, Accuracy: 1.00
[INFO] [PRIMARY] Running validation for the primary task.
[INFO] [PRIMARY] Validation Loss: 0.7776, Accuracy: 0.7

[INFO] [PRIMARY] Iter: 1, Epoch: 15, Loss: 0.1394, Accuracy: 1.00
[INFO] [PRIMARY] Running validation for the primary task.
[INFO] [PRIMARY] Validation Loss: 0.6500, Accuracy: 0.81: 
[INFO] [PRIMARY] No improvement in validation accuracy.
[INFO] [PRIMARY] Iter: 1, Epoch: 16, Loss: 0.1350, Accuracy: 1.00
[INFO] [PRIMARY] Running validation for the primary task.
[INFO] [PRIMARY] Validation Loss: 0.2892, Accuracy: 0.91: 
[INFO] [PRIMARY] Saving the model parameters
[INFO] [PRIMARY] Fully optimized, stopping early.

[INFO] [SECONDARY] Training for the secondary task.
[INFO] [SECONDARY] Iter: 2, Epoch: 0, Loss: 0.2228, Accuracy: 1.00
[INFO] [SECONDARY] Running validation for the secondary task.
[INFO] [SECONDARY] Validation Loss: 0.0115, Accuracy: 1.00: 
[INFO] [SECONDARY] Fully optimized, stopping early.

[INFO] [PRIMARY] Training for the primary task.
[INFO] [PRIMARY] Iter: 2, Epoch: 0, Loss: 1.3312, Accuracy: 0.75
[INFO] [PRIMARY] Running validation for the primary task.
[INFO] [PRIMARY]

[INFO] [SECONDARY] Iter: 6, Epoch: 0, Loss: 0.2342, Accuracy: 1.00
[INFO] [SECONDARY] Running validation for the secondary task.
[INFO] [SECONDARY] Validation Loss: 0.0067, Accuracy: 1.00: 
[INFO] [SECONDARY] Fully optimized, stopping early.

[INFO] [PRIMARY] Training for the primary task.
[INFO] [PRIMARY] Iter: 6, Epoch: 0, Loss: 1.6215, Accuracy: 0.78
[INFO] [PRIMARY] Running validation for the primary task.
[INFO] [PRIMARY] Validation Loss: 0.8333, Accuracy: 0.72: 
[INFO] [PRIMARY] No improvement in validation accuracy.
[INFO] [PRIMARY] Iter: 6, Epoch: 1, Loss: 0.2555, Accuracy: 1.00
[INFO] [PRIMARY] Running validation for the primary task.
[INFO] [PRIMARY] Validation Loss: 0.6072, Accuracy: 0.84: 
[INFO] [PRIMARY] No improvement in validation accuracy.
[INFO] [PRIMARY] Iter: 6, Epoch: 2, Loss: 0.1332, Accuracy: 1.00
[INFO] [PRIMARY] Running validation for the primary task.
[INFO] [PRIMARY] Validation Loss: 0.7363, Accuracy: 0.84: 
[INFO] [PRIMARY] No improvement in validation accur